In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("Data/robotsensors.csv")

In [3]:
df.head()

,UDI,ProductID,Type,Airtemperature,Processtemperature,Rotationalspeed,Torque,Toolwear,FailureType
0,1,M14860,M,298.1,308.6,1551,42.8,0,NoFailure
1,2,L47181,L,298.2,308.7,1408,46.3,3,NoFailure
2,3,L47182,L,298.1,308.5,1498,49.4,5,NoFailure
3,4,L47183,L,298.2,308.6,1433,39.5,7,NoFailure
4,5,L47184,L,298.2,308.7,1408,40.0,9,NoFailure


In [4]:
df.shape

(10000, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   UDI                 10000 non-null  int64  
 1   ProductID           10000 non-null  object 
 2   Type                10000 non-null  object 
 3   Airtemperature      10000 non-null  float64
 4   Processtemperature  10000 non-null  float64
 5   Rotationalspeed     10000 non-null  int64  
 6   Torque              10000 non-null  float64
 7   Toolwear            10000 non-null  int64  
 8   FailureType         10000 non-null  object 
dtypes: float64(3), int64(3), object(3)
memory usage: 703.2+ KB


In [6]:
df.isnull().sum()

UDI                   0
ProductID             0
Type                  0
Airtemperature        0
Processtemperature    0
Rotationalspeed       0
Torque                0
Toolwear              0
FailureType           0
dtype: int64

In [7]:
df.describe()

,UDI,Airtemperature,Processtemperature,Rotationalspeed,Torque,Toolwear
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,300.004930,310.005560,1538.776100,39.986910,107.951000
std,2886.89568,2.000259,1.483734,179.284096,9.968934,63.654147
min,1.00000,295.300000,305.700000,1168.000000,3.800000,0.000000
25%,2500.75000,298.300000,308.800000,1423.000000,33.200000,53.000000
50%,5000.50000,300.100000,310.100000,1503.000000,40.100000,108.000000
75%,7500.25000,301.500000,311.100000,1612.000000,46.800000,162.000000
max,10000.00000,304.500000,313.800000,2886.000000,76.600000,253.000000


In [8]:
df['FailureType'].value_counts()

FailureType
NoFailure                 9652
HeatDissipationFailure     112
PowerFailure                95
OverstrainFailure           78
ToolWearFailure             45
RandomFailure               18
Name: count, dtype: int64

In [9]:
X = df.drop(columns=['FailureType', 'ProductID', 'Type', 'UDI'], axis=1)
Y = df['FailureType']

In [10]:
print(X)

      Airtemperature  Processtemperature  Rotationalspeed  Torque  Toolwear
0              298.1               308.6             1551    42.8         0
1              298.2               308.7             1408    46.3         3
2              298.1               308.5             1498    49.4         5
3              298.2               308.6             1433    39.5         7
4              298.2               308.7             1408    40.0         9
...              ...                 ...              ...     ...       ...
9995           298.8               308.4             1604    29.5        14
9996           298.9               308.4             1632    31.8        17
9997           299.0               308.6             1645    33.4        22
9998           299.0               308.7             1408    48.5        25
9999           299.0               308.7             1500    40.2        30

[10000 rows x 5 columns]


In [11]:
print(Y)

0       NoFailure
1       NoFailure
2       NoFailure
3       NoFailure
4       NoFailure
          ...    
9995    NoFailure
9996    NoFailure
9997    NoFailure
9998    NoFailure
9999    NoFailure
Name: FailureType, Length: 10000, dtype: object


In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y,random_state=2)

In [13]:
print(X.shape, X_train.shape, X_test.shape)

(10000, 5) (8000, 5) (2000, 5)


In [14]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from imblearn.combine import SMOTETomek

In [16]:
smk = SMOTETomek(random_state=42)
X_smk, Y_smk = smk.fit_resample(X_train,Y_train)

In [17]:
X_smk.shape, Y_smk.shape

((46260, 5), (46260,))

In [18]:
Y_train.value_counts()

FailureType
NoFailure                 7722
HeatDissipationFailure      90
PowerFailure                76
OverstrainFailure           62
ToolWearFailure             36
RandomFailure               14
Name: count, dtype: int64

In [19]:
Y_smk.value_counts()

FailureType
OverstrainFailure         7719
HeatDissipationFailure    7716
PowerFailure              7713
RandomFailure             7713
ToolWearFailure           7711
NoFailure                 7688
Name: count, dtype: int64

In [20]:
model = LogisticRegression(max_iter=5000)

In [21]:
model.fit(X_smk.values, Y_smk.values)

c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=5000)

In [22]:
X_train_prediction = model.predict(X_smk.values)
training_data_accuracy = accuracy_score(X_train_prediction, Y_smk)

In [23]:
print('Accuracy on Training data : ', training_data_accuracy )

Accuracy on Training data :  0.8296584522265457


In [24]:
X_test_prediction = model.predict(X_test.values)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [25]:
print('Accuracy on Testing data : ', testing_data_accuracy )

Accuracy on Testing data :  0.5315


In [26]:
input_data = (398.8,109.0,1268,40.8,105)

input_data_asnumpyarray = np.asarray(input_data)
input_data_reshaped = input_data_asnumpyarray.reshape(1,-1)

prediction = model.predict(input_data_reshaped)
print(prediction)

if(prediction[0]==0):
  print("The machine is not with fault")
else:
  print("The machine is with fault")

['HeatDissipationFailure']
The machine is with fault


In [31]:
import joblib

In [32]:
joblib.dump(model, 'classifier2')

['classifier2']